## Setup autoreload, warnings and helper functions

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, Markdown
def print_heading(string):
    display(Markdown(f"# {string}"))
def print_subheading(string):
    display(Markdown(f"## {string}"))

## Set the visibility of cuda devices (in case your system contains more than one)

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
# %cd ../..
# %ls

In [4]:
def save_pickle(x, file):
    with open(file, 'wb') as f_file:
        pickle.dump(x, f_file, protocol=4)
        
def load_pickle(file):
    with open(file, 'rb') as f_file:
        result = pickle.load(f_file)
    return result


In [5]:
import torch

print(torch.__version__)

1.10.0+cu111


/ekaterina/env/norppa/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Imports

In [6]:
%matplotlib inline
import torch
from config_whaleshark import config
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import zipfile
import tensorflow as tf
import wget
import pickle
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from torchvision.datasets.utils import download_url
from datasets import SimpleDataset, DatasetSlice

from tools import apply_pipeline, crop_step, curry, curry_sequential, apply_pipeline_dataset, get_save_step, apply_sequential, compose, compose_sequential
from tonemapping.tonemapping import tonemap, tonemap_step
from segmentation.segmentation import segment
from pattern_extraction.extract_pattern import extract_pattern
from pattern_extraction.extract_pattern import smart_resize
from reidentification.identify import encode_single, encode_pipeline, encode_dataset, identify, identify_single, apply_geometric, encode_patches, extract_patches
from reidentification.visualisation import visualise_match
from reidentification.find_matches import find_matches

2023-06-14 10:19:21.373690: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-14 10:19:22.947716: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-14 10:19:22.993193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2023-06-14 10:19:22.993236: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-14 10:19:22.996457: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-14 10:19:22.996518: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library li

## Create a configuration file.
You can change the default parameters in config.py

In [77]:
cfg = config()

## Download the dataset

## Create dataset variables

In [78]:
cfg["topk"]=20

In [9]:
def resize_dataset(input, size):
    image, img_label = input
    if image is None:
        return [input]

    result, ratio = smart_resize(image, size, return_ratio=True)
    img_label["resize_ratio"] = ratio
    return [(result, img_label)]

In [15]:
def get_topk_matrix(identification_result):
    result = []
    for (db_labels, q_labels) in identification_result:
        q_class = q_labels['class_id']
        q_ln = len(q_labels['labels'])
        result.append([db_label['db_label']['class_id']==q_class for db_label in db_labels]*q_ln)
    return np.asarray(result)


def get_topk_accuracy(identification_result):
    result = []
    for (db_labels, q_labels) in identification_result:
        q_class = q_labels['class_id']
        q_ln = len(q_labels['labels'])
        result.append([db_label['db_label']['class_id']==q_class for db_label in db_labels]*q_ln)
    result = np.asarray(result)
    return [sum((np.sum(result[:, :j+1], axis=1) > 0)) / len(result) for j in range(result.shape[1])]

def print_topk_accuracy(identification_result, label=""):
    topk_acc = get_topk_accuracy(identification_result)
    print(label)
    for (i, acc) in enumerate(topk_acc):
        print(f"Top-{i+1} accuracy: {acc*100}%")
    return identification_result


In [12]:
import kornia as K
import kornia.feature as KF
from kornia_moons.feature import *

def deaffinize_laf(laf):
    xy = KF.laf.get_laf_center(laf)
    sc = KF.laf.get_laf_scale(laf)
    ori = KF.laf.get_laf_orientation(laf)
    return KF.laf.laf_from_center_scale_ori(xy, sc, ori)

def get_lafs_and_descriptors(img, harrisz_kpts,
                             kornia_descriptor,
                             ori,
                             aff,
                             aff_string = 'def',
                             mrSize=1.0,
                             num_feats = 2048, device=torch.device('cuda')):
    # We will not train anything, so let's save time and memory by no_grad()
    with torch.no_grad():
        timg = K.image_to_tensor(img, False).float()/255.
        timg = timg.to(device)
#         timg = K.color.rgb_to_grayscale(timg)
        # timg = K.color.rgb_to_grayscale(K.image_to_tensor(img, False))/255.
        # timg = timg.to(device)
        hz_pt = torch.from_numpy(harrisz_kpts).float()
        lafs = KF.laf.ellipse_to_laf(hz_pt[None])
        lafs[0,:,0,0] = hz_pt[:,2] * mrSize
        lafs[0,:,0,1] = hz_pt[:,3] * mrSize
        lafs[0,:,1,0] = hz_pt[:,3] * 0
        lafs[0,:,1,1] = hz_pt[:,4] * mrSize
        lafs = lafs.to(timg.device)
        # border image changed from 5 to 10 !!!
        good_lafs_mask = KF.laf.laf_is_inside_image(lafs, timg, 10)
        # all taken!!!
        # good_lafs_mask[:] = True 
        good_lafs = lafs[good_lafs_mask][None]
        if aff_string == 'def':
            out_lafs = good_lafs
        elif aff_string == 'no':
            out_lafs = deaffinize_laf(good_lafs)
        elif aff_string == 'affnet':
            out_lafs = deaffinize_laf(good_lafs)
            out_lafs = aff(out_lafs,timg)
        elif aff_string == 'orinet_affnet':
            out_lafs = deaffinize_laf(good_lafs)
            out_lafs = ori(aff(out_lafs,timg),timg)
        else:
            raise ValueError('Unknown affine str')
        out_lafs = out_lafs[:,:num_feats]
        # We will estimate affine shape of the feature and re-orient the keypoints with the OriNet
        patches = KF.extract_patches_from_pyramid(timg,out_lafs, 32)
        B, N, CH, H, W = patches.size()
        # Descriptor accepts standard tensor [B, CH, H, W], while patches are [B, N, CH, H, W] shape
        # So we need to reshape a bit :) 
        descs = kornia_descriptor(patches.view(B * N, CH, H, W)).view(B * N, -1)
    return out_lafs, descs.detach().cpu().numpy()#, good_lafs_mask[0].detach().cpu().numpy()

In [101]:

from extract_patches.laf import LAFs2ell
from extract_patches.core import extract_patches as keypoints_to_patches
from torchvision import transforms
from tqdm import tqdm

import scipy.io
import os
import cv2

# import h5py

# f = h5py.File('/ekaterina/work/src/NORPPA/repository/NORPPA/temp/files/result_pattern_tonemapped_wrong.mat','r')
# data = f.get('data/result')
# data = np.array(data)

train_path = "/ekaterina/work/data/whaleshark_wrong_tonemapped_mini"
# train_path = "/ekaterina/work/data/whaleshark_norppa_tonemapped_resized/train"
dataset_train = Path(train_path)
train_dataset = SimpleDataset(dataset_train)


def get_class_and_name(full_path):
    head, file = os.path.split(full_path)
    _, class_name = os.path.split(head)
    return os.path.join(class_name, file)

def cell_to_dict(mat):
    res = {}
    for i in range(mat.shape[0]):
        res[os.path.join(mat[i, 0][0], mat[i, 1][0])] = mat[i, 2]
    return res


def patchify_load(dataset, file, config):
    mat = scipy.io.loadmat(file)
    matdict = cell_to_dict(mat['result'])
    
    net = config["net"]
    result = []
    labels = []
    inds = []
    all_ells = []
    ind = 0
    num_files = len(dataset)
    dataset_transforms = transforms.Grayscale(num_output_channels=1)
    
    if config['use_cuda']:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    PS = 32
    descriptor = KF.HardNet(True)
#     descriptor = KF.HardNet8(True)        
    
    descriptor = descriptor.to(device)
    # print (device)
    descriptor.eval()
    aff_est = KF.LAFAffNetShapeEstimator(True).to(device)
    orienter = KF.LAFOrienter(32, angle_detector=KF.OriNet(True)).to(device)
    orienter.eval()
    aff_est.eval()
    NUM_KP = 500
    mrSize = 0.5
    
    for i, (image, img_label) in enumerate(tqdm(dataset)):
        if image is None:
            all_ells.append(None)
            labels.append(img_label)
            continue
        image = dataset_transforms(image)
        num_files-=1
        if sum(image.getextrema()) == 0:
            all_ells.append(None)
            labels.append(img_label)
            continue
#         patches, ells = patch_extraction(image, config)
        key = get_class_and_name(img_label['file'])
        if key not in matdict:
            continue

        hz_kpts = matdict[key]
        
        lafs, descs = get_lafs_and_descriptors(np.array(image), hz_kpts,
                     descriptor,
                     orienter,
                     aff_est,
                     aff_string = "affnet", # "orinet_affnet", # 
                     mrSize=mrSize,
                     num_feats = NUM_KP, device=device)
        kps_back = opencv_kpts_from_laf(lafs, mrSize)
#         out_img = cv2.drawKeypoints(np.array(image), kps_back, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#         plt.imshow(out_img)
#         plt.show()
        
#         print(descs.shape)
        patch_features = descs
#         ells = LAFs2ell(np.array(lafs[0].cpu()))
        ells = np.array([[kp.pt[0], kp.pt[1], kp.size, kp.size, kp.angle] for kp in kps_back])
        

        all_ells.append(ells)
        inds.extend([i] * patch_features.shape[0])
        labels.append(img_label)
        result.append(patch_features)

    labels = np.array(labels)
    return np.vstack(result), np.array(inds), labels, all_ells

def extract_patches_load(dataset, file, config):
    return (dataset, patchify_load(dataset, file, config))

keypoints_path = '/ekaterina/work/src/NORPPA/repository/NORPPA/temp/files/result_tonemapped_wrong.mat'


def patchify_test(dataset, config):
    net = config["net"]
    result = []
    labels = []
    inds = []
    all_ells = []
    ind = 0
    num_files = len(dataset)
    dataset_transforms = transforms.Grayscale(num_output_channels=1)
    
    if config['use_cuda']:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
        
#     descriptor = KF.HardNet8(True)        
    mrSize = 1.0
#     detector = KF.KeyNetAffNetHardNet(num_features=5000, upright=False, device=device, scale_laf=mrSize)
    detector = KF.DISK.from_pretrained('depth', device=device)
    for i, (image, img_label) in enumerate(tqdm(dataset)):
        if image is None:
            all_ells.append(None)
            labels.append(img_label)
            continue
#         image = dataset_transforms(image)
        num_files-=1
#         if sum(image.getextrema()) == 0:
#             all_ells.append(None)
#             labels.append(img_label)
#             continue
#         image = np.array(image)[None, :, :, None]
        image = np.array(image)[None, :, :]
        timg = K.image_to_tensor(image, False).float()/255.
        timg = timg.to(device)
        
#         lafs, _, descs = detector(timg)        

#         kps_back = opencv_kpts_from_laf(lafs, mrSize)
#         patch_features = descs.cpu()[0, ]
#         ells = np.array([[kp.pt[0], kp.pt[1], kp.size, kp.size, kp.angle] for kp in kps_back])
        
        disk = detector(timg, pad_if_not_divisible=True)[0]
        patch_features = disk.descriptors.cpu().numpy()
        pts = disk.keypoints.cpu().numpy()
        ells = np.array([[kp[0], kp[1], 5, 5, 0] for kp in pts])
        

        all_ells.append(ells)
        inds.extend([i] * patch_features.shape[0])
        labels.append(img_label)
        result.append(patch_features)

    labels = np.array(labels)
    return np.vstack(result), np.array(inds), labels, all_ells
def extract_patches_test(dataset, config):
    return (dataset, patchify_test(dataset, config))

pipeline = [
#     curry(extract_patches_load, file=keypoints_path, config=cfg),
    curry(extract_patches_test, config=cfg),
    curry(encode_patches, cfg=cfg)
]

encoded_train_dataset = apply_pipeline_dataset(train_dataset, pipeline, verbose=True)


100%|███████████████████████████████████████████| 99/99 [00:01<00:00, 50.63it/s]


Completed 1/2 steps
Calculating PCA


/ekaterina/work/src/NORPPA/repository/NORPPA/reidentification/identify.py:225: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_ells = np.array(all_ells)


Getting encoding parameters...
Encoding...
Completed 2/2 steps


In [102]:

test_pipeline1 = [
                 curry(identify, encoded_train_dataset, cfg["topk"], leave_one_out=True),
                 curry(print_topk_accuracy, label="Before geometric verification:"),
                ]
test_pipeline2 = [
                 curry_sequential(find_matches, cfg),
                 curry_sequential(apply_geometric, cfg["geometric"]),
                 curry(print_topk_accuracy, label="After geometric verification:"),
#                  curry_sequential(visualise_match, cfg["topk"])
                ]

matches1 = apply_pipeline_dataset(encoded_train_dataset, test_pipeline1)
None

20


100%|████████████████████████████████████████| 99/99 [00:00<00:00, 25488.68it/s]

Before geometric verification:
Top-1 accuracy: 13.131313131313133%
Top-2 accuracy: 28.28282828282828%
Top-3 accuracy: 37.37373737373738%
Top-4 accuracy: 42.42424242424242%
Top-5 accuracy: 48.484848484848484%
Top-6 accuracy: 57.57575757575758%
Top-7 accuracy: 60.60606060606061%
Top-8 accuracy: 61.61616161616161%
Top-9 accuracy: 64.64646464646465%
Top-10 accuracy: 69.6969696969697%
Top-11 accuracy: 74.74747474747475%
Top-12 accuracy: 76.76767676767676%
Top-13 accuracy: 81.81818181818183%
Top-14 accuracy: 84.84848484848484%
Top-15 accuracy: 85.85858585858585%
Top-16 accuracy: 85.85858585858585%
Top-17 accuracy: 85.85858585858585%
Top-18 accuracy: 86.86868686868688%
Top-19 accuracy: 89.8989898989899%
Top-20 accuracy: 89.8989898989899%


In [ ]:
matches2 = apply_pipeline_dataset(matches1, test_pipeline2)

# with open("tonemapped_pattern_whaleshark_scale2_matches.pickle", 'wb') as f_file:
#     pickle.dump(codebooks, f_file, protocol=4)
# None

# Scale 2
# Top-1 accuracy: 31.0%
# Top-2 accuracy: 49.0%
# Top-3 accuracy: 55.00000000000001%
# Top-4 accuracy: 59.0%
# Top-5 accuracy: 66.0%

# Scale 2 (no resize)
# Top-1 accuracy: 26.0%
# Top-2 accuracy: 43.0%
# Top-3 accuracy: 55.00000000000001%
# Top-4 accuracy: 61.0%
# Top-5 accuracy: 68.0%

# Scale 1
# Top-1 accuracy: 22.0%
# Top-2 accuracy: 26.0%
# Top-3 accuracy: 42.0%
# Top-4 accuracy: 49.0%
# Top-5 accuracy: 52.0%

# Scale 3
# Top-1 accuracy: 26.0%
# Top-2 accuracy: 41.0%
# Top-3 accuracy: 53.0%
# Top-4 accuracy: 62.0%
# Top-5 accuracy: 70.0%


# Scale 1
# Before geometric verification:
# Top-1 accuracy: 27.0%
# Top-2 accuracy: 41.0%
# Top-3 accuracy: 48.0%
# Top-4 accuracy: 55.00000000000001%
# Top-5 accuracy: 56.00000000000001%

# After geometric verification:
# Top-1 accuracy: 42.0%
# Top-2 accuracy: 51.0%
# Top-3 accuracy: 61.0%
# Top-4 accuracy: 66.0%
# Top-5 accuracy: 71.0%

# Scale 1 tonemapped + tonemapped codebooks

# Before GV
# Top-1 accuracy: 41.0%
# Top-2 accuracy: 55.00000000000001%
# Top-3 accuracy: 57.99999999999999%
# Top-4 accuracy: 63.0%
# Top-5 accuracy: 71.0%

# After GV
# Top-1 accuracy: 47.0%
# Top-2 accuracy: 59.0%
# Top-3 accuracy: 64.0%
# Top-4 accuracy: 69.0%
# Top-5 accuracy: 73.0%

# KeyNetAffNetHardNet tonemapped mini

# Before GV
# Top-1 accuracy: 19.19191919191919%
# Top-2 accuracy: 27.27272727272727%
# Top-3 accuracy: 35.35353535353536%
# Top-4 accuracy: 42.42424242424242%
# Top-5 accuracy: 48.484848484848484%

# After GV
# Top-1 accuracy: 36.36363636363637%
# Top-2 accuracy: 43.43434343434344%
# Top-3 accuracy: 53.535353535353536%
# Top-4 accuracy: 57.57575757575758%
# Top-5 accuracy: 59.59595959595959%


# HarrisZ+ mini tonemapped

# Before GV
# Top-1 accuracy: 23.232323232323232%
# Top-2 accuracy: 31.313131313131315%
# Top-3 accuracy: 43.43434343434344%
# Top-4 accuracy: 49.494949494949495%
# Top-5 accuracy: 51.515151515151516%

# After GV
# Top-1 accuracy: 24.242424242424242%
# Top-2 accuracy: 34.34343434343434%
# Top-3 accuracy: 42.42424242424242%
# Top-4 accuracy: 47.474747474747474%
# Top-5 accuracy: 55.55555555555556%

# HarrisZ+ full tonemapped

# Before GV
# Top-1 accuracy: 36.14346459604363%
# Top-2 accuracy: 42.42928452579035%
# Top-3 accuracy: 46.12682566093548%
# Top-4 accuracy: 48.73359216121279%
# Top-5 accuracy: 51.02606766500277%

# After GV

# Top-1 accuracy: 47.402477352560545%
# Top-2 accuracy: 52.09835459419486%
# Top-3 accuracy: 54.85302273987798%
# Top-4 accuracy: 56.83120724718063%
# Top-5 accuracy: 58.402662229617306%

In [ ]:
curry_sequential(visualise_match, 5)(matches2[:5])

In [ ]:
matches, query_label = matches2[0]

def get_label(lb):
    return lb['labels'][0]['class_id']

def get_db_label(lb):
    return get_label(lb['db_label'])

get_db_label(matches[0])

get_label(query_label)

wrong_matches = [i for (i, (matches, query_label)) in enumerate(matches2) if not any([get_db_label(match)==get_label(query_label) for match in matches[:5]])]



[autoreload of torch failed: Traceback (most recent call last):
  File "/ekaterina/env/norppa/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/ekaterina/env/norppa/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/ekaterina/env/norppa/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/ekaterina/env/norppa/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/ekaterina/env/norppa/lib/python3.7/site-packages/torch/__init__.py", line 22, in <module>
    from ._utils import _import_dotted_name, classproperty
ImportError: cannot import name

In [ ]:
# wrong_matches[0][1]

# hard_labels = set([get_label(wrong_match[1]) for wrong_match in wrong_matches])

# hard_labels

In [ ]:
# len(wrong_matches)

# hard_dataset = [train_dataset[i] for i in wrong_matches]

In [ ]:
# save_pickle(wrong_matches, "wrong_match_inds.pickle")

In [ ]:
train_dataset[0][1]

In [ ]:
len(train_dataset)

In [ ]:
len([(img, label) for (img, label) in train_dataset if label['class_id'] in hard_labels])